# Finetune RuGPTs in megatron and deepspeed
How to finetune RuGPTs models with megatron and deepspeed. Example for RuGPT3Small. Note for other models it will take more GPU memory.

This notebook is valid for all RuGPTs models except RuGPT3XL.
## Install env

In [ ]:
# !pip3 install transformers==3.5.0

In [ ]:
# %%writefile setup.sh

# export CUDA_HOME=/usr/local/cuda-10.1
# git clone https://github.com/NVIDIA/apex
# pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
# !sh setup.sh

In [ ]:
# !git clone  https://github.com/sberbank-ai/ru-gpts

In [ ]:
# !pip install deepspeed==0.3.7

## Download files

In [ ]:
# !wget -O train.txt https://www.dropbox.com/s/oa3v9c7g9bp40xw/train.txt?dl=0
# !wget -O valid.txt https://www.dropbox.com/s/mworl3ld6r3bg62/valid.txt?dl=0

In [14]:
with open("/notebooks/js_script_prechunks.json", "r") as ptj:
  sources = ptj.readlines()

from collections import defaultdict
from tqdm import tqdm

chunks = defaultdict(list)
num = 0

# for l in tqdm(sources[:500]):
for l in tqdm(sources[:1000]):
  
  
  if '+++++\n' in l:
    num +=1
  else:
    chunks[num].append(l)
  
  first_line = chunks.get(num)
  if first_line is not None:
    l_splits = first_line[0].split(" ") 
    # for s in l_splits:
    #   print (f"\t\t========{s}")
  
  # with open("/content/trial.txt", "a+") as f:
  with open("/notebooks/sberbank_rugpts/huge_dataset_2.txt", "a+") as f:
    for j in range(len(chunks)):
      for i in range(max(len(chunks[j]),3)):
        prompt = str(''.join(chunks[j][:i]))
        # a reasonable chunk of code for completeion  
        lng = 5
        completion = str(''.join(chunks[j][i:i+lng]))
        if not (prompt == '' or completion == ''):
          f.write(f"{prompt}\n {completion}\n")
      first_line = chunks.get(j)
      if first_line is not None:
        l_splits = first_line[0].split(" ") 
        for s in range(len(l_splits)):
          # print (f"\t\t========{l_splits[s]}")
          prompt = str(" ".join(l_splits[:s]))
          completion =  str(" ".join(l_splits[s:]))
          if not (prompt == '' or completion == ''):
            f.write(f"{prompt}\n {completion}\n")

100%|██████████| 1000/1000 [00:02<00:00, 378.01it/s]


In [19]:
# Get our dataset
# !gdown https://drive.google.com/file/d/1-0K7OdvgX9AuPbJakCaAWgpL5qCSyuB9
# !cp -v /content/drive/Shareddrives/Machine_/ivolution_dataset/huge_dataset.txt /content/trial.txt
# !wc -l /content/drive/Shareddrives/Machine_/ivolution_dataset/huge_dataset.txt
# !wc -l /content/drive/MyDrive/huge_dataset.txt
# !wc -l /content/huge_dataset.txt  
# !wc -l /notebooks/sberbank_rugpts/huge_dataset_2.txt  
print(int(8927077*0.9), int(8927077*0.1))
# print(int(10847600*0.9), int(10847600*0.1))
# input ("Use that?")

# Train/test split
380253500-342228150-38025350
# !head -n 8034369 /content/huge_dataset.txt > /content/train.txt
# !tail -892707 /content/huge_dataset.txt > /content/valid.txt
# !head -n 9762840  /notebooks/sberbank_rugpts/huge_dataset_2.txt > /notebooks/sberbank_rugpts/train.txt
# !tail -1084760 /notebooks/sberbank_rugpts/huge_dataset.txt > /notebooks/sberbank_rugpts/valid.txt
!head -n 8034369  /notebooks/sberbank_rugpts/huge_dataset_2.txt > /notebooks/sberbank_rugpts/train.txt
!tail -892707 /notebooks/sberbank_rugpts/huge_dataset_2.txt > /notebooks/sberbank_rugpts/valid.txt

8034369 892707


## Prepare data for parallel
We use custom implementation of distributed dataset. For training and evaluating we should specify file `file.list` with list of paths to txt files. All files from `file.list` will be splitted between aviable GPUs. The logic of splitting is described by the following code:

```python
shard_size = len(files) // world_size
shard_start = rank * shard_size
shard_end = (rank + 1) * shard_size
files = files[shard_start:shard_end]
```

For more details please see full code of dataset: `src.dataset_rugpt3.RuGpt3TextDataset`.

In [21]:
# !echo train.txt > train.list
# !echo valid.txt > valid.list


!echo /notebooks/sberbank_rugpts/train.txt > /notebooks/sberbank_rugpts/train.list
!echo /notebooks/sberbank_rugpts/valid.txt > /notebooks/sberbank_rugpts/valid.list

## Train
Load model from Huggingface and finetune on essays.

This will take arount ten minutes.

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:${HOME}/sberbank_rugpts

%cd  /notebooks/sberbank_rugpts
    
# !USE_DEEPSPEED=1 python -m torch.distributed.launch --nproc_per_node 1 pretrain_gpt3.py \
#   --train-data-path "/notebooks/train.list" \
#   --val-data-path "/notebooks/valid.list" \
#   --test-data-path "/notebooks/valid.list" \
#   --max-files-per-process 100 \
#   --logging-dir="our_logs" \
#   --save our_model \
#   --load-huggingface sberbank-ai/rugpt3large_based_on_gpt2 \
#   --save-interval 10 \
#   --log-interval 10 \
#   --eval-interval 10 \
#   --eval-iters 10 \
#   --model-parallel-size 1 \
#   --num-layers 24 \
#   --hidden-size 1536 \
#   --num-attention-heads 16 \
#   --batch-size 1 \
#   --seq-length 2048 \
#   --max-position-embeddings 2048 \
#   --train-iters 10 \
#   --resume-dataloader \
#   --distributed-backend "nccl" \
#   --lr 0.00015 \
#   --lr-decay-style "cosine" \
#   --lr-decay-iters 3200 \
#   --clip-grad 0.5 \
#   --warmup .004 \
#   --fp16 \
#   --checkpoint-activations \
#   --deepspeed-activation-checkpointing \
#   --deepspeed \
#   --deepspeed_config /notebooks/sberbank_rugpts/src/deepspeed_config/our_gpt3_large_2048.json\
#   --make-vocab-size-divisible-by 1


print ("Trying to free some GPU memory")
import torch
torch.cuda.empty_cache()


!USE_DEEPSPEED=1 python -m torch.distributed.launch --nproc_per_node 1 pretrain_gpt3.py \
  --train-data-path "/notebooks/train.list" \
  --val-data-path "/notebooks/valid.list" \
  --test-data-path "/notebooks/valid.list" \
  --max-files-per-process 100 \
  --logging-dir="our_logs" \
  --save our_model \
  --load-huggingface sberbank-ai/rugpt3large_based_on_gpt2 \
  --save-interval 10 \
  --log-interval 10 \
  --eval-interval 10 \
  --eval-iters 100 \
  --model-parallel-size 1 \
  --num-layers 24 \
  --hidden-size 1536 \
  --num-attention-heads 16 \
  --batch-size 1 \
  --seq-length 2048 \
  --max-position-embeddings 2048 \
  --train-iters 1000 \
  --resume-dataloader \
  --distributed-backend "nccl" \
  --lr 0.00015 \
  --lr-decay-style "cosine" \
  --lr-decay-iters 3200 \
  --clip-grad 0.5 \
  --warmup .004 \
  --fp16 \
  --checkpoint-activations \
  --deepspeed-activation-checkpointing \
  --deepspeed \
  --deepspeed_config /notebooks/sberbank_rugpts/src/deepspeed_config/our_gpt3_large_2048.json \
  --make-vocab-size-divisible-by 1 \
  --load our_model

# No such file or directory: 'our_model/iter_0002000/mp_rank_00/model_optim_rng.pt'

/notebooks/sberbank_rugpts
Trying to free some GPU memory
using world size: 1 and model-parallel size: 1 
 > using dynamic loss scaling
> initializing model parallel with size 1
[2021-04-16 09:04:59,139] [INFO] [checkpointing.py:629:_configure_using_config_file] {'partition_activations': False, 'contiguous_memory_optimization': False, 'cpu_checkpointing': False, 'number_checkpoints': None, 'synchronize_checkpoint_boundary': False, 'profile': False}
Pretrain GPT3 model
arguments:
  attention_dropout ............ 0.1
  num_attention_heads .......... 16
  hidden_size .................. 1536
  intermediate_size ............ None
  num_layers ................... 24
  layernorm_epsilon ............ 1e-05
  hidden_dropout ............... 0.1
  max_position_embeddings ...... 2048
  vocab_size ................... 30522
  deep_init .................... False
  make_vocab_size_divisible_by . 1
  cpu_optimizer ................ False
  cpu_torch_adam ............... False
  sparse_mode ............

At the end of training output should be something like this:

"-----------------------------------------------------------------------------------------

 validation loss at the end of training for test data | LM loss: 3.0002 | LM PPL: 20.090

-----------------------------------------------------------------------------------------"

## Generate

Load pretrained model from dir and generate.

In [2]:
!nvidia-smi

Thu Apr 15 08:33:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 450.36.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        On   | 00000000:00:05.0 Off |                  Off |
| 26%   32C    P8     6W / 180W |      1MiB / 16278MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
!export PYTHONPATH=${PYTHONPATH}:${HOME}/sberbank_rugpts
# !pwd
%cd /notebooks/sberbank_rugpts

print ("Trying to free some GPU memory")
import torch
torch.cuda.empty_cache()

# !python generate_samples.py \
#   --load model/ \
#   --model-parallel-size 1 \
#   --num-layers 12 \
#   --hidden-size 768 \
#   --num-attention-heads 12 \
#   --batch-size 1 \
#   --seq-length 500 \
#   --max-position-embeddings 2048 \
#   --distributed-backend "nccl" \
#   --tokenizer-path sberbank-ai/rugpt3small_based_on_gpt2 \
#   --no-load-optim

# !python generate_samples.py \
!USE_DEEPSPEED=1 python -m torch.distributed.launch generate_samples.py \
  --load our_model/ \
  --model-parallel-size 1 \
  --num-layers 24 \
  --hidden-size 1536 \
  --num-attention-heads 16 \
  --batch-size 1 \
  --seq-length 2048 \
  --max-position-embeddings 2048 \
  --distributed-backend "nccl" \
  --tokenizer-path sberbank-ai/rugpt3large_based_on_gpt2 \
  --deepspeed \
  --deepspeed_config /notebooks/sberbank_rugpts/src/deepspeed_config/our_gpt3_large_2048_nofp16.json \
  --make-vocab-size-divisible-by 1
# --no-load-optim \

/notebooks/sberbank_rugpts
Trying to free some GPU memory
Generate Samples
using world size: 1 and model-parallel size: 1 
 > using dynamic loss scaling
in main, we've got args.tokenizer_path sberbank-ai/rugpt3large_based_on_gpt2
> initializing model parallel with size 1
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234
we've got args.tokenizer_path sberbank-ai/rugpt3large_based_on_gpt2
prepare tokenizer done, size 50257
building GPT3 model ...
asserting we have a correct number of attention heads...
we're using deepspeed, and so we're getting a sparse attention config
Calling GPT3Model constructor...
 > number of parameters on model parallel rank 0: 760300032
placing the model on device 0
Setting up distributed training...
Using sberbank magic DDP
ready to return model
Load checkpoint from our_model/
load_checkpoint has deepspeed True
loading a deepspeed checkpoint...
u

### Convert checkpoint to Huggingface format

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:${HOME}/sberbank_rugpts
%cd /notebooks/sberbank_rugpts
#     !python ru-gpts/convert2huggingface.py \
!python convert2huggingface.py \
  --load our_model/ \
  --model-parallel-size 1 \
  --num-layers 24 \
  --hidden-size 1536 \
  --num-attention-heads 16 \
  --max-position-embeddings 2048 \
  --tokenizer-path sberbank-ai/rugpt3large_based_on_gpt2 \
  --no-load-optim \
  --export-huggingface our_model_hf


#### Test load

In [ ]:
from transformers import GPT2LMHeadModel

In [ ]:
model = GPT2LMHeadModel.from_pretrained("model_hf")